In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from dateutil.relativedelta import relativedelta
from functools import reduce
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics




%matplotlib inline

In [2]:
attend_codes = pd.read_csv("did_not_attend_codes.txt", sep = '\t')
attend_codes = attend_codes['medcode'].astype(str).values.tolist()

weight_loss_codes = pd.read_csv("weight_loss_codes.txt", sep = '\t')
weight_loss_codes = weight_loss_codes['medcode'].astype(str).values.tolist()

third_codes = pd.read_csv("third_party_consultation.txt", sep = '\t')
third_codes = third_codes['medcode'].astype(str).values.tolist()







va_codes = pd.read_csv("violent_agressive_behaviour.txt", sep = '\t')
va_codes = va_codes['medcode'].astype(str).values.tolist()

bc_codes = pd.read_csv("behaviour_change.txt", sep = '\t')
bc_codes = bc_codes['medcode'].astype(str).values.tolist()

dw_codes = pd.read_csv("disorientation_wandering.txt", sep = '\t')
dw_codes = dw_codes['medcode'].astype(str).values.tolist()

pc_codes = pd.read_csv("personality_change.txt", sep = '\t')
pc_codes = pc_codes['medcode'].astype(str).values.tolist()

behav_codes = list(set(va_codes + bc_codes + dw_codes + pc_codes))









ch_codes = pd.read_csv("care_in_home.txt", sep = '\t')
ch_codes = ch_codes['medcode'].astype(str).values.tolist()

nh_codes = pd.read_csv("needs_help.txt", sep = '\t')
nh_codes = nh_codes['medcode'].astype(str).values.tolist()

rhc_codes = pd.read_csv("receives_home_care.txt", sep = '\t')
rhc_codes = rhc_codes['medcode'].astype(str).values.tolist()

ssi_codes = pd.read_csv("social_services_interaction.txt", sep = '\t')
ssi_codes = ssi_codes['medcode'].astype(str).values.tolist()

interaction_codes = list(set(ch_codes + nh_codes + rhc_codes + ssi_codes))



Getting rid of any cases without any matched controls

In [3]:
matches = pd.read_csv('Matching_File.txt',delimiter='\t')
matches = matches.dropna(how='any') 

Taking the matched file and creating a vertical database 'cases_and_controls'

In [4]:
cases = matches[['case_patid','control_patid','case_index','case_gender','case_birthyear']]
cases['isCase'] = True
controls = cases[['control_patid','case_gender','case_birthyear','case_index']]
controls = controls.rename(index=str,columns={'control_patid':'patid','case_gender':'gender','case_birthyear':'yob','case_index':'index_date'})
controls['isCase']=False
cases = cases.drop(['control_patid'], axis=1)
cases = cases.rename(index=str, columns={'case_patid':'patid','case_gender':'gender','case_birthyear':'yob','case_index':'index_date'})
cases_and_controls = pd.concat([cases,controls])
controls['patid'].dropna()
controls['patid'] = controls['patid'].astype('float64')
controls['patid'].apply(lambda x: x if np.isnan(x) else np.int64(x))
controls['patid'].dropna()
print()

/Users/Joanne/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/Joanne/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Making 'index_date' date_time

In [16]:
cases_and_controls['index_date'] = pd.to_datetime(cases_and_controls['index_date'])

Dataframe with all appointment information

In [17]:
df = pd.read_csv('medcoded_entries.csv', usecols =['patid','eventdate', 'medcode'])
df['medcode'] = df['medcode'].astype(str)
df['medcode'].str.strip()

df = df

Joining all eventdate with index_date and then working out the difference between the two

In [18]:
df['eventdate'] = pd.to_datetime(df['eventdate'])

In [19]:
df = df.merge(cases_and_controls, how='left', on=['patid'])

In [20]:
df['time_difference'] = (df['eventdate'].subtract(df['index_date']))/np.timedelta64(1, 'Y')

To exclude all patients with less than 5 years of data

In [21]:
def isLess(x):
    if x <= -5: return True
    else: return False

dft = df
dft['5yr'] = df['time_difference'].apply(isLess)

dft = dft[dft['5yr'] == True]

new_cc = cases_and_controls

new_cc['5yr_Boo'] = new_cc.patid.isin(dft['patid'])

new_cc = new_cc[new_cc['5yr_Boo'] == True]

len(new_cc.axes[0])

94557

In [28]:
matches = pd.read_csv('Matching_File.txt',delimiter='\t')
matches = matches.dropna(how='any') 
matches['5yr'] = matches.case_patid.isin(new_cc['patid'])
matches = matches[matches['5yr'] == True]
cases = matches[['case_patid','control_patid','case_index','case_gender','case_birthyear']]
cases['isCase'] = True
controls = cases[['control_patid','case_gender','case_birthyear','case_index']]
controls = controls.rename(index=str,columns={'control_patid':'patid','case_gender':'gender','case_birthyear':'yob','case_index':'index_date'})
controls['isCase']=False
cases = cases.drop(['control_patid'], axis=1)
cases = cases.rename(index=str, columns={'case_patid':'patid','case_gender':'gender','case_birthyear':'yob','case_index':'index_date'})
cases_and_controls = pd.concat([cases,controls])
controls['patid'].dropna()
controls['patid'] = controls['patid'].astype('float64')
controls['patid'].apply(lambda x: x if np.isnan(x) else np.int64(x))
controls['patid'].dropna()

/Users/Joanne/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/Joanne/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if sys.path[0] == '':


0         4355328.0
1         6290216.0
2         2770420.0
3         5473429.0
4         5951081.0
5        17742438.0
6         5262373.0
7         6394024.0
8        15830371.0
9         9053629.0
10        2042198.0
11        8669429.0
12        9380236.0
13         150510.0
14        4145573.0
15       11375168.0
16         809598.0
17        1592003.0
18        4976153.0
19        3672153.0
20       12542457.0
21        5927320.0
22        5938496.0
23        2388116.0
24        6613271.0
25        5044625.0
26        3646191.0
28       14431632.0
29       17317066.0
30       12911385.0
            ...    
47828     2974035.0
47829     1385115.0
47830     7533113.0
47831     4570504.0
47832     2999375.0
47833      522462.0
47834     3798652.0
47835    68401229.0
47836     4352135.0
47837    11233173.0
47838     1008666.0
47839     4836628.0
47840    16025205.0
47841     7367091.0
47842     3389483.0
47843     1584096.0
47844    23324287.0
47845     2158117.0
47846     3488544.0


In [31]:
len(cases.axes[0]) + len(controls.axes[0])

94528

To exclude all cases without a dementia code

In [36]:
df_jra = pd.read_csv('1_key_data.csv',sep = '\t')
df_dem = cases_and_controls.merge(df_jra, on='patid', how='outer')


def noDementia(x):
    if x == 0: return False
    else: return True
    
dft = df_dem[df_dem['isCase'] == True]

dft['noDem'] = df_dem['dementia_codes'].apply(noDementia)

dft = dft[dft['noDem'] == True]

new_cc = cases_and_controls

new_cc['noDem'] = new_cc.patid.isin(dft['patid'])

new_cc = new_cc[new_cc['noDem'] == True]

len(new_cc.axes[0])

/Users/Joanne/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


47209

In [37]:
matches['Dem'] = matches.case_patid.isin(new_cc['patid'])
matches = matches[matches['Dem'] == True]
cases = matches[['case_patid','control_patid','case_index','case_gender','case_birthyear']]
cases['isCase'] = True
controls = cases[['control_patid','case_gender','case_birthyear','case_index']]
controls = controls.rename(index=str,columns={'control_patid':'patid','case_gender':'gender','case_birthyear':'yob','case_index':'index_date'})
controls['isCase']=False
cases = cases.drop(['control_patid'], axis=1)
cases = cases.rename(index=str, columns={'case_patid':'patid','case_gender':'gender','case_birthyear':'yob','case_index':'index_date'})
cases_and_controls = pd.concat([cases,controls])
controls['patid'].dropna()
controls['patid'] = controls['patid'].astype('float64')
controls['patid'].apply(lambda x: x if np.isnan(x) else np.int64(x))
controls['patid'].dropna()


print(len(cases.axes[0]) + len(controls.axes[0]))

/Users/Joanne/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/Joanne/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


94418


In [38]:
df_jra = pd.read_csv('1_key_data.csv',sep = '\t')
df_dem = cases_and_controls.merge(df_jra, on='patid', how='outer')


def noDementia(x):
    if x == 0: return False
    else: return True
    
dft = df_dem[df_dem['isCase'] == False]

dft['noDem2'] = df_dem['dementia_codes'].apply(noDementia)

dft = dft[dft['noDem2'] == False]

new_cc = cases_and_controls

new_cc['noDem2'] = new_cc.patid.isin(dft['patid'])

new_cc = new_cc[new_cc['noDem2'] == True]


/Users/Joanne/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [39]:
matches['noDem'] = matches.control_patid.isin(new_cc['patid'])
matches = matches[matches['noDem'] == True]
cases = matches[['case_patid','control_patid','case_index','case_gender','case_birthyear']]
cases['isCase'] = True
controls = cases[['control_patid','case_gender','case_birthyear','case_index']]
controls = controls.rename(index=str,columns={'control_patid':'patid','case_gender':'gender','case_birthyear':'yob','case_index':'index_date'})
controls['isCase']=False
cases = cases.drop(['control_patid'], axis=1)
cases = cases.rename(index=str, columns={'case_patid':'patid','case_gender':'gender','case_birthyear':'yob','case_index':'index_date'})
cases_and_controls = pd.concat([cases,controls])
controls['patid'].dropna()
controls['patid'] = controls['patid'].astype('float64')
controls['patid'].apply(lambda x: x if np.isnan(x) else np.int64(x))
controls['patid'].dropna()


print(len(cases.axes[0]) + len(controls.axes[0]))

/Users/Joanne/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/Joanne/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


94330


In [40]:
df_codes = cases_and_controls
def names(x, codes, y='column_name', df=df, df_codes= df_codes):
#x = The name of the symptom/ what the columns are to be called
#codes = the list of medcodes corresponding to the symptoms
#y = the name of the array of names for the columns created below


#Creating the names of the variables where x is replaces with the symptom
    y = [x + '_yr_neg5', x + '_yr_neg4', x + '_yr_neg3', x + '_yr_neg2',
        x + '_yr_neg1', x + '_yr_0']

#does the name as for y but this is the nontime dependant version
    codes_name = x + '_codes'
    
    
#Finding all of the codes relating to that given variable    
    df[codes_name] = df['medcode'].apply(lambda xx: 1 if (xx in codes) else 0)
    
#Splitting the codes into each year for the cases    
    for i in range(6):
        z =  df[(df['time_difference'] > (i-6)) & (df['time_difference'] < i-5)]
        z = z.groupby('patid').sum().reset_index()
        z = z[['patid', codes_name]]
        z.columns = ['patid', y[i]]
        df_codes = pd.merge(df_codes,
                 z[['patid', y[i]]],
                 on='patid', 
                 how='left')
    
    return df_codes

In [41]:
df_codes_atten = names('attend_codes_', attend_codes, 'attend_codes_names')

In [42]:
df_codes_tp = names('third_party', third_codes, 'third_party_names')


In [43]:
df_codes_bc = names('behav_change', behav_codes, 'behav_change_names')


In [44]:
df_codes_i = names('interaction', interaction_codes, 'interaction_names')


In [45]:
df_codes_w = names('weight_loss', weight_loss_codes, 'weight_loss_names')


In [46]:
dfs = [df_codes_atten, df_codes_tp, df_codes_bc, df_codes_i, df_codes_w] 

df_final = reduce(lambda left,right: pd.merge(left,right,on=['gender', 'index_date', 'isCase','patid', 'yob']), dfs)

In [47]:
print(len(df_final.axes[0]))

94330


In [48]:
df_final.to_csv("patients_timeseries.csv")

In [57]:
df_final = pd.read_csv('patients_timeseries.csv',delimiter=',')

In [60]:
df_final['year'] = df['index_date'].map(lambda x: x.year)

In [59]:
df_final['year']

0        2006
1        2006
2        2006
3        2006
4        2006
5        2006
6        2006
7        2006
8        2006
9        2006
10       2006
11       2006
12       2006
13       2006
14       2006
15       2006
16       2006
17       2006
18       2006
19       2006
20       2006
21       2006
22       2006
23       2006
24       2006
25       2006
26       2006
27       2006
28       2006
29       2006
         ... 
94300    2008
94301    2008
94302    2008
94303    2008
94304    2008
94305    2008
94306    2008
94307    2008
94308    2008
94309    2008
94310    2008
94311    2008
94312    2008
94313    2008
94314    2008
94315    2008
94316    2008
94317    2008
94318    2008
94319    2008
94320    2008
94321    2008
94322    2008
94323    2004
94324    2004
94325    2004
94326    2004
94327    2004
94328    2004
94329    2004
Name: year, Length: 94330, dtype: int64